# Importing Drive 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
spath = "/content/drive/MyDrive/COVID-19_Radiography_Dataset"

# Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import os

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

import seaborn as sns

from sklearn.metrics import classification_report, confusion_matrix
import glob
import matplotlib.pyplot as plt
import cv2

# Data and Preprocessing

In [ ]:
train = "/content/drive/MyDrive/COVID-19_Radiography_Dataset/Train" 
val ="/content/drive/MyDrive/COVID-19_Radiography_Dataset/Val"
test = "/content/drive/MyDrive/COVID-19_Radiography_Dataset/Test"

gen_train = ImageDataGenerator(rescale = 1./255., zoom_range = 0.2,)
gen_val = ImageDataGenerator(rescale = 1./255.)
gen_test = ImageDataGenerator(rescale = 1./255.)

train_gen = gen_train.flow_from_directory(train, batch_size = 64, class_mode = 'categorical', target_size = (220, 220))
validation_gen = gen_val.flow_from_directory(val, batch_size = 64, class_mode = 'categorical', target_size=(220, 220))
test_gen = gen_test.flow_from_directory(test, shuffle = False, batch_size=64, class_mode = 'categorical', target_size=(220, 220))

Found 5786 images belonging to 2 classes.
Found 722 images belonging to 2 classes.
Found 722 images belonging to 2 classes.


# Model Training

**Model Flow** 


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=(220, 220, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dropout(rate=0.1),
    tf.keras.layers.Dense(units=2, activation='softmax')
])

In [ ]:
model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=["accuracy"])

# Early Stopping to avoid overfitting of problem (Avoiding Overfitting strategy-2) 
callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=2,restore_best_weights=True)

# Here we have taken less steps per epoch to create randomness for model as well as to reduce epoch training time as there are many images in each batch 
history = model.fit(train_gen, validation_data = validation_gen, steps_per_epoch = 40, epochs = 11, callbacks = callback)

Epoch 1/11
40/40 [==============================] - 594s 15s/step - loss: 0.6833 - accuracy: 0.6023 - val_loss: 0.4010 - val_accuracy: 0.8269
Epoch 2/11
40/40 [==============================] - 310s 8s/step - loss: 0.5518 - accuracy: 0.7203 - val_loss: 0.3236 - val_accuracy: 0.8823
Epoch 3/11
40/40 [==============================] - 302s 7s/step - loss: 0.4443 - accuracy: 0.8031 - val_loss: 0.3166 - val_accuracy: 0.8504
Epoch 4/11
40/40 [==============================] - 286s 7s/step - loss: 0.4466 - accuracy: 0.7910 - val_loss: 0.3146 - val_accuracy: 0.8643
Epoch 5/11
40/40 [==============================] - 289s 7s/step - loss: 0.3737 - accuracy: 0.8315 - val_loss: 0.2747 - val_accuracy: 0.8684
Epoch 6/11
40/40 [==============================] - 283s 7s/step - loss: 0.3703 - accuracy: 0.8331 - val_loss: 0.2604 - val_accuracy: 0.8837
Epoch 7/11
40/40 [==============================] - 305s 8s/step - loss: 0.3088 - accuracy: 0.8684 - val_loss: 0.2355 - val_accuracy: 0.8892
Epoch 8/11
4

In [ ]:
history1 = model.fit(train_gen, validation_data = validation_gen, steps_per_epoch = 40, epochs = 5, callbacks = callback)

Epoch 1/5
40/40 [==============================] - 286s 7s/step - loss: 0.2641 - accuracy: 0.8875 - val_loss: 0.1512 - val_accuracy: 0.9432
Epoch 2/5
40/40 [==============================] - 289s 7s/step - loss: 0.2503 - accuracy: 0.8910 - val_loss: 0.1126 - val_accuracy: 0.9681
Epoch 3/5
40/40 [==============================] - 283s 7s/step - loss: 0.2452 - accuracy: 0.9078 - val_loss: 0.1549 - val_accuracy: 0.9349
Epoch 4/5
40/40 [==============================] - 282s 7s/step - loss: 0.2305 - accuracy: 0.9044 - val_loss: 0.0995 - val_accuracy: 0.9765
Epoch 5/5
40/40 [==============================] - 303s 8s/step - loss: 0.2141 - accuracy: 0.9105 - val_loss: 0.1526 - val_accuracy: 0.9571


# Saving the model parameters

In [ ]:
model.save("CNN.h5")

In [ ]:
history=np.load("CNN-history.npy",allow_pickle=True).item()
history1=np.load("CNN-history1.npy",allow_pickle=True).item()


In [ ]:
np.save("CNN-history.npy",history2)